In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

## Data preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_train=pd.read_csv('/content/drive/MyDrive/AML_assignment_2/UNSW_NB15_training-set.csv')
data_test=pd.read_csv('/content/drive/MyDrive/AML_assignment_2/UNSW_NB15_testing-set.csv')

In [4]:
data=pd.concat([data_train,data_test],ignore_index=True)
y=data['label']
data.drop('label',axis=1,inplace=True)

### Exploring metrics

In [5]:
unique_cat={}
cat_features=[]
for column in data.columns:
  if len(data[column].unique())<=12:
    unique_cat[column]=data[column].value_counts()/data.shape[0]
  if type(data[column][3])==str:
    cat_features.append(column)


In [6]:
for key in unique_cat:
  print(unique_cat[key])

FIN    0.454700
INT    0.451883
CON    0.078138
REQ    0.014875
RST    0.000326
ECO    0.000047
ACC    0.000016
CLO    0.000004
PAR    0.000004
URN    0.000004
no     0.000004
Name: state, dtype: float64
0      0.467410
252    0.310172
29     0.217807
60     0.004238
30     0.000167
31     0.000147
253    0.000035
32     0.000016
254    0.000008
Name: dttl, dtype: float64
2    0.450781
1    0.297233
0    0.224455
6    0.014491
3    0.012927
4    0.000109
5    0.000004
Name: ct_state_ttl, dtype: float64
0    0.987407
1    0.012493
4    0.000062
2    0.000039
Name: is_ftp_login, dtype: float64
0    0.987399
1    0.012477
2    0.000062
4    0.000062
Name: ct_ftp_cmd, dtype: float64
0     0.901348
1     0.091604
4     0.005138
9     0.000838
2     0.000357
16    0.000186
6     0.000163
12    0.000140
30    0.000116
25    0.000097
3     0.000012
Name: ct_flw_http_mthd, dtype: float64
0    0.985726
1    0.014274
Name: is_sm_ips_ports, dtype: float64
Normal            0.360923
Generic        

We can see that for 'is_sm_ips_ports', 'ct_ftp_cmd', 'is_ftp_login' more than 98% of column equals to specififc value. So we can remove these features. Also we exclude 'proto' because it contains more than 130 unique 

In [7]:
constant_columns=['is_sm_ips_ports', 'ct_ftp_cmd', 'is_ftp_login','proto']
data.drop(columns=constant_columns,inplace=True)


In [8]:
cat_features.remove('proto')

### Fill missing values

In [9]:
from sklearn.impute import SimpleImputer
for column in data.columns[1:]: 
  imp_mean = SimpleImputer( strategy='most_frequent')
  imp_mean.fit(np.array(data[column]).reshape(-1,1))
  data[column]=imp_mean.transform(np.array(data[column]).reshape(-1,1))



In [10]:
# from sklearn.preprocessing import LabelEncoder
# for cat in cat_features:
#   encoder=LabelEncoder()

#   data[cat]=encoder.fit_transform(data[cat])

data=pd.get_dummies(data,columns=cat_features)

In [11]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
data = scaler.fit_transform(data)


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( data, y, test_size=0.2, random_state=42)

## Conditional GAN

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:

## Undercomplete
class Generator(nn.Module):
    def __init__(self, data_size,n_classes=2):
      super(Generator, self).__init__()

      self.label_emb = nn.Embedding(n_classes, n_classes)
      input_size=n_classes + data_size
      self.hidden_layer1 = nn.Sequential(
          nn.Linear(input_size, 128),
          nn.LeakyReLU(0.2)

      )

      self.hidden_layer2 = nn.Sequential(
          nn.Linear(128, 256),
          nn.LeakyReLU(0.2)

      )
      self.hidden_layer3 = nn.Sequential(
          nn.Linear(256, 512),
          nn.LeakyReLU(0.2),

      )
      self.hidden_layer4 = nn.Sequential(
          nn.Linear(512, 34),
          nn.Tahn()

      )

        

        
    def forward(self, data,labels,n_classes=2):

      gen_input = torch.cat((self.label_emb(labels), data), -1)
      x = get_input
      output=self.hidden_layer1(x)
      output=self.hidden_layer2(output)
      output=self.hidden_layer3(output)
      output=self.hidden_layer4(output)
      return output.to(device)
        
        
    
class Discriminator(nn.Module):
    def __init__(self, data_size,n_classes):

      super(Discriminator, self).__init__()
      self.label_emb = nn.Embedding(n_classes, n_classes)
      # Step 1 : Define the encoder 
      # Step 2 : Define the decoder
      # Step 3 : Initialize the weights (optional)
      input_size=n_classes + data_size
      
      self.hidden_layer1 = nn.Sequential(
          nn.Linear(input_size, 512),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.3)

      )

      self.hidden_layer2 = nn.Sequential(
          nn.Linear(512, 256),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.3)

      )
      self.hidden_layer3 = nn.Sequential(
          nn.Linear(256, 128),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.3)

      )
      self.hidden_layer4 = nn.Sequential(
          nn.Linear(128,1),
          nn.Sigmoid()

      )

        

        
    def forward(self, data,labels):
      # Step 1: Pass the input through encoder to get latent representation
      # Step 2: Take latent representation and pass through decoder
      gen_input = torch.cat((self.label_emb(labels), data), -1)
      x = get_input
      output=self.hidden_layer1(x)
      output=self.hidden_layer2(output)
      output=self.hidden_layer3(output)
      output=self.hidden_layer4(output)
      return output.to(device)
        


In [15]:
batchSize = 50
learning_rate = 0.0005
num_epochs = 2000
input_dim=
generator=Generator()
discriminator=Discriminator()
gen_optimizer=torch.optim.SGD(generator.parameters(),lr=learning_rate)
dis_optimizer=torch.optim.SGD(discriminator.parameters(),lr=learning_rate)

# print(summary(AE,input_size=(1, 64)))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(AE.parameters(),lr = learning_rate)

#Create a random dataset
data_loader = DataLoader(TensorDataset(torch.from_numpy(full_train)),batch_size=batchSize,shuffle=True)
# del full_train

TypeError: ignored

In [ ]:
losses=[]
for epoch in range(num_epochs):
  epoch_loss = 0.0
  for X in data_loader:
    X = X[0].to(device)

    optimizer.zero_grad()
    # forward
    output = AE(X.float())
    loss = criterion(output*X.float()[1], X.float()[0])
    #print(output.dtype)
    # backward
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  losses.append(epoch_loss)
  # log
  #print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))